In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import sqlite3
import pandas as pd
import os

# 1. Connecting to the SQLite database
conn = sqlite3.connect('Patient_Related_DB.db')

# 2. Define the base path to your Google Drive folder
# Based on your screenshots, the path should look like this:
base_path = '/content/drive/MyDrive/CSI 660_LAB DATA data'

# 3. Load your 4 CSVs using the Drive path
# If the path above is slightly different, you can right-click the file
# in the Colab file browser (left side) and select "Copy path".
pd.read_csv(os.path.join(base_path, 'Patient.csv')).to_sql('Patient', conn, if_exists='replace', index=False)
pd.read_csv(os.path.join(base_path, 'procedure_types.csv')).to_sql('procedure_types', conn, if_exists='replace', index=False)
pd.read_csv(os.path.join(base_path, 'procedure.csv')).to_sql('procedure', conn, if_exists='replace', index=False)
pd.read_csv(os.path.join(base_path, 'patients_procedure.csv')).to_sql('patients_procedure', conn, if_exists='replace', index=False)

print("Database Created successfully from Google Drive files!")

# 4. Example verification query
query = "SELECT patient_id, (2026 - birth_year) AS Age FROM Patient LIMIT 5"
df = pd.read_sql(query, conn)
display(df)

Database Created successfully from Google Drive files!


,patient_id,Age
0,PT001,50
1,PT002,27
2,PT003,48
3,PT004,39
4,PT005,61


In [14]:
# --- STEP 1: CREATE THE VIEWS ---
views = {
    "patient_directory": "SELECT patient_id, first_name, last_name FROM Patient",
    "young_patients": "SELECT * FROM Patient WHERE birth_year > 1990",
    "avg_height_by_sex": "SELECT sex, AVG(height_cm) AS avg_height FROM Patient GROUP BY sex"
}

for name, query in views.items():
    conn.execute(f"DROP VIEW IF EXISTS {name}")
    conn.execute(f"CREATE VIEW {name} AS {query}")

In [15]:

# --- STEP 2: PRACTICE ACTIVITIES (The DML part) ---

# Activity 1: View everyone's first/last names (Slide 6)
print("--- Activity 1: Patient Directory ---")
df1 = pd.read_sql("SELECT * FROM patient_directory LIMIT 5", conn)
display(df1)

--- Activity 1: Patient Directory ---


,patient_id,first_name,last_name
0,PT001,Olivia,Brown
1,PT002,Emily,Martinez
2,PT003,Olivia,Hall
3,PT004,Ethan,Brown
4,PT005,Emily,Brown


In [16]:

# Activity 3: View Calculated Data (Slide 5)
print("\n--- Activity 3: Average Height by Sex ---")
df3 = pd.read_sql("SELECT * FROM avg_height_by_sex", conn)
display(df3)


--- Activity 3: Average Height by Sex ---


,sex,avg_height
0,Female,169.210000
1,Male,169.448889
2,Other,171.766667


In [17]:
female_patient_view =[
    "DROP VIEW IF EXISTS female_patient_view",
    "CREATE VIEW female_patient_view AS SELECT * FROM Patient WHERE sex = 'Female'",
]

In [10]:
# --- STEP 1: DEFINE AND CREATE VIEWS ---
view_commands = [
    "DROP VIEW IF EXISTS female_patient_view",
    "CREATE VIEW female_patient_view AS SELECT * FROM Patient WHERE sex = 'Female'",

    "DROP VIEW IF EXISTS lifescience_procedures",
    "CREATE VIEW lifescience_procedures AS SELECT * FROM procedure_types WHERE procedure_type IN ('Diagnostic', 'Therapeutic')",

    "DROP VIEW IF EXISTS patient_public_list",
    "CREATE VIEW patient_public_list AS SELECT first_name, last_name FROM Patient"
]

for cmd in view_commands:
    conn.execute(cmd)

# --- STEP 2: VERIFY AND DISPLAY (DML) ---
print("--- Result: Female Patient View ---")
display(pd.read_sql("SELECT * FROM female_patient_view LIMIT 5", conn))

print("\n--- Result: Life Science Procedures ---")
display(pd.read_sql("SELECT * FROM lifescience_procedures", conn))

print("\n--- Result: Patient Public List (No IDs) ---")
display(pd.read_sql("SELECT * FROM patient_public_list LIMIT 5", conn))

--- Result: Female Patient View ---


,patient_id,first_name,last_name,birth_year,sex,height_cm,weight_kg
0,PT004,Ethan,Brown,1987,Female,158.4,88.7
1,PT009,Lucas,Hall,1983,Female,164.4,103.8
2,PT015,Mia,Walker,1965,Female,151.9,81.4
3,PT017,William,Smith,1967,Female,183.3,105.7
4,PT018,Michael,Garcia,1992,Female,157.1,96.0



--- Result: Life Science Procedures ---


,procedure_code,procedure_name,typical_duration_min,procedure_type
0,DX001,Chest X-Ray,15,Diagnostic
1,DX002,MRI Brain,60,Diagnostic
2,DX003,CT Abdomen,45,Diagnostic
3,DX004,Blood Panel,10,Diagnostic
4,DX005,Electrocardiogram,30,Diagnostic
5,TH001,Physical Therapy,50,Therapeutic
6,TH002,Vaccination,10,Therapeutic
7,TH003,Dialysis,180,Therapeutic
8,TH004,Chemotherapy,120,Therapeutic
9,TH005,Respiratory Therapy,40,Therapeutic



--- Result: Patient Public List (No IDs) ---


,first_name,last_name
0,Olivia,Brown
1,Emily,Martinez
2,Olivia,Hall
3,Ethan,Brown
4,Emily,Brown


In [11]:
# 1. View of a View (Slide 7)
# Using our previously created 'female_patient_view' to find 'Young Females'
conn.execute("""
CREATE VIEW IF NOT EXISTS young_female_view AS
SELECT * FROM female_patient_view
WHERE birth_year > 1990;
""")

# 2. Modifying a View (OR REPLACE)
# Let's redefine the 'patient_public_list' to include their birth year
conn.execute("""
CREATE VIEW IF NOT EXISTS patient_public_list AS
SELECT first_name, last_name, birth_year
FROM Patient;
""")

# 3. Demonstration
print("--- Result: Young Female View (Nested) ---")
display(pd.read_sql("SELECT * FROM young_female_view", conn))

print("\n--- Result: Modified Public List ---")
display(pd.read_sql("SELECT * FROM patient_public_list LIMIT 5", conn))

--- Result: Young Female View (Nested) ---


,patient_id,first_name,last_name,birth_year,sex,height_cm,weight_kg
0,PT018,Michael,Garcia,1992,Female,157.1,96.0
1,PT023,Olivia,Garcia,2004,Female,154.3,108.5
2,PT024,Noah,Robinson,2002,Female,173.2,47.8
3,PT033,Ethan,Lee,2005,Female,170.0,69.4
4,PT035,Emily,Hall,1999,Female,189.9,45.8
5,PT043,Henry,Thompson,1997,Female,161.0,67.1
6,PT050,Sophia,Harris,2003,Female,190.3,56.4
7,PT051,Noah,Thomas,2001,Female,168.1,84.0
8,PT052,Alex,Thomas,2003,Female,182.1,52.6
9,PT057,Henry,Robinson,2005,Female,168.8,103.2



--- Result: Modified Public List ---


,first_name,last_name
0,Olivia,Brown
1,Emily,Martinez
2,Olivia,Hall
3,Ethan,Brown
4,Emily,Brown


In [18]:
# Creating a small subset to match the slide example
query = """
SELECT p.patient_id, p.last_name, pp.procedure_id
FROM Patient p
INNER JOIN patients_procedure pp ON p.patient_id = pp.patient_id
WHERE p.patient_id IN ('PT001', 'PT003', 'PT041')
"""

print("--- The Joined Result ---")
display(pd.read_sql(query, conn))

--- The Joined Result ---


,patient_id,last_name,procedure_id
0,PT001,Brown,PR0183
1,PT003,Hall,PR0002
2,PT003,Hall,PR0143
3,PT003,Hall,PR0152
4,PT003,Hall,PR0158
5,PT003,Hall,PR0298
6,PT041,Johnson,PR0001
7,PT041,Johnson,PR0118
8,PT041,Johnson,PR0146
9,PT041,Johnson,PR0162


In [19]:
# Activity 1: Inner Join (Clinical Activity Report)
print("--- Result: Patients with Procedures (Inner Join) ---")
inner_query = """
SELECT p.first_name, p.last_name, pr.procedure_name, pr.procedure_year
FROM Patient p
INNER JOIN patients_procedure pp ON p.patient_id = pp.patient_id
INNER JOIN procedure pr ON pp.procedure_id = pr.procedure_id
LIMIT 10;
"""
display(pd.read_sql(inner_query, conn))

# Activity 2: Left Join (Gap Analysis - Lab Q6)
print("\n--- Result: Patients with NO Procedures (Left Join + NULL) ---")
gap_query = """
SELECT p.first_name, p.last_name, pp.procedure_id
FROM Patient p
LEFT JOIN patients_procedure pp ON p.patient_id = pp.patient_id
WHERE pp.procedure_id IS NULL;
"""
display(pd.read_sql(gap_query, conn))

--- Result: Patients with Procedures (Inner Join) ---


,first_name,last_name,procedure_name,procedure_year
0,Olivia,Brown,Knee Arthroscopy,2015
1,Olivia,Hall,Hernia Repair,2016
2,Olivia,Hall,Blood Panel,2020
3,Olivia,Hall,Cataract Surgery,2020
4,Olivia,Hall,Gallbladder Removal,2023
5,Olivia,Hall,Hernia Repair,2021
6,Ethan,Brown,Electrocardiogram,2018
7,Emily,Brown,Cardiac Surgery,2024
8,Emily,Brown,Electrocardiogram,2018
9,Sophia,Anderson,Chest X-Ray,2017



--- Result: Patients with NO Procedures (Left Join + NULL) ---


,first_name,last_name,procedure_id
0,Emily,Martinez,None
1,William,Smith,None
2,Daniel,Martin,None
3,Lucas,White,None
4,Amelia,Lee,None
5,Daniel,Brown,None
6,Robert,Smith,None
7,Henry,Johnson,None
8,Amelia,Martin,None
9,Sophia,Thomas,None


In [12]:
# Cartesian Product: Every patient paired with every possible procedure type
# We do not use a 'WHERE' or 'ON' clause here (Slide 122)
query = """
SELECT p.first_name, p.last_name, pt.procedure_name
FROM Patient p, procedure_types pt
LIMIT 15;
"""

print("--- The Cartesian Product Result (First 15 of many) ---")
df_cartesian = pd.read_sql(query, conn)
display(df_cartesian)

# Calculation for the students (Slide 117)
total_patients = pd.read_sql("SELECT COUNT(*) FROM Patient", conn).iloc[0,0]
total_types = pd.read_sql("SELECT COUNT(*) FROM procedure_types", conn).iloc[0,0]
print(f"\nMathematical Result: {total_patients} patients x {total_types} tests = {total_patients * total_types} total rows!")

--- The Cartesian Product Result (First 15 of many) ---


,first_name,last_name,procedure_name
0,Olivia,Brown,Chest X-Ray
1,Olivia,Brown,MRI Brain
2,Olivia,Brown,CT Abdomen
3,Olivia,Brown,Blood Panel
4,Olivia,Brown,Electrocardiogram
5,Olivia,Brown,Physical Therapy
6,Olivia,Brown,Vaccination
7,Olivia,Brown,Dialysis
8,Olivia,Brown,Chemotherapy
9,Olivia,Brown,Respiratory Therapy



Mathematical Result: 120 patients x 20 tests = 2400 total rows!


In [20]:
# 2. Activity: UNION
# Identify patients who are either "Heavy" (>100kg) OR "Tall" (>180cm)
print("--- Result: UNION (Heavy OR Tall) ---")
union_query = """
SELECT first_name, last_name, weight_kg, height_cm
FROM Patient WHERE weight_kg > 100
UNION
SELECT first_name, last_name, weight_kg, height_cm
FROM Patient WHERE height_cm > 180;
"""
df_union = pd.read_sql(union_query, conn)
display(df_union.head())

# 3. Activity: INTERSECT
# Identify patients who are BOTH "Heavy" AND "Tall"
print("\n--- Result: INTERSECT (Heavy AND Tall) ---")
intersect_query = """
SELECT first_name, last_name, weight_kg, height_cm
FROM Patient WHERE weight_kg > 100
INTERSECT
SELECT first_name, last_name, weight_kg, height_cm
FROM Patient WHERE height_cm > 180;
"""
df_intersect = pd.read_sql(intersect_query, conn)
display(df_intersect)

# 4. Activity: EXCEPT / MINUS
# Find patients who are "Heavy" (>100kg) but NOT "Tall" (<= 180cm)
print("\n--- Result: EXCEPT (Heavy but NOT Tall) ---")
except_query = """
SELECT first_name, last_name, weight_kg, height_cm
FROM Patient WHERE weight_kg > 100
EXCEPT
SELECT first_name, last_name, weight_kg, height_cm
FROM Patient WHERE height_cm > 180;
"""
df_except = pd.read_sql(except_query, conn)
display(df_except.head())

# 5. Activity: HAVING Clause
# Identify "High-Utilizers" by filtering aggregated groups
print("\n--- Result: HAVING (Patients with > 3 procedures) ---")
having_query = """
SELECT patient_id, COUNT(*) as procedure_count
FROM patients_procedure
GROUP BY patient_id
HAVING COUNT(*) > 3
ORDER BY procedure_count DESC;
"""
df_having = pd.read_sql(having_query, conn)
display(df_having)

# Close connection if done
# conn.close()

--- Result: UNION (Heavy OR Tall) ---


,first_name,last_name,weight_kg,height_cm
0,Alex,Thomas,52.6,182.1
1,Amelia,Harris,104.2,152.3
2,Amelia,Walker,72.7,194.6
3,Amelia,White,65.1,186.1
4,Benjamin,Robinson,48.5,181.7



--- Result: INTERSECT (Heavy AND Tall) ---


,first_name,last_name,weight_kg,height_cm
0,Emily,Clark,110.1,185.0
1,Emily,White,105.5,186.3
2,Henry,Martinez,115.9,182.6
3,Isabella,Lewis,109.5,189.3
4,Linda,Williams,109.9,182.0
5,Michael,Johnson,116.7,183.4
6,Noah,Brown,108.4,186.3
7,Olivia,Williams,107.2,181.7
8,Robert,Martin,109.5,186.0
9,Sophia,Thomas,100.5,185.5



--- Result: EXCEPT (Heavy but NOT Tall) ---


,first_name,last_name,weight_kg,height_cm
0,Amelia,Harris,104.2,152.3
1,Benjamin,Thompson,113.8,165.4
2,Charlotte,Anderson,110.4,158.1
3,Charlotte,Robinson,116.5,153.0
4,Daniel,Brown,101.9,157.3



--- Result: HAVING (Patients with > 3 procedures) ---


,patient_id,procedure_count
0,PT049,8
1,PT060,6
2,PT057,6
3,PT041,6
4,PT023,6
5,PT022,6
6,PT107,5
7,PT091,5
8,PT076,5
9,PT067,5
